# Atomic Systems

## 1. Importing/installing atomman

This cell imports atomman if it is already installed, or it installs it.  This makes it possible to run the Notebook in a colab environment.

__NOTE__: If atomman gets installed, the Notebook kernel must be restarted and the cell executed again before running the remaining cells!

In [1]:
try:
    # Check if potentials has been installed
    import atomman as am

except:
    # Install if need be and print message
    !pip install atomman
    print('!!!!! RESTART KERNEL/RUNTIME TO USE POTENTIALS !!!!!')

else:
    # Print version of the loaded atomman package
    print(f'atomman version {am.__version__} loaded')

atomman version 1.4.0 loaded


## 2. Unit conversion basics

Units are handled in atomman by converting values to compatible working units when they are set, and then retrieving the results in the units that you wish.  This method avoids any overhead associated with tracking units during calculations, but does mean that invalid unit conversions are not explicitly checked.

- atomman.unitconvert.set_in_units(value, unit) sets a value in the specified units by converting to the compatible working units (i.e. when you assign values). 
- atomman.unitconvert.get_in_units(value, unit) gets a value in the specified units by converting from the compatible working units (i.e. when you retrieve values).

The default working units for atomman are
- length = ‘angstrom’ = ‘Å’
- mass = ‘amu’
- energy = ‘eV’
- charge = ‘e’
- temperature = ‘K’

In [4]:
import atomman.unitconvert as uc

In [5]:
# Set value with complex units
p = uc.set_in_units(205, 'kg/(m*s^2)')

print('P in working units is', p)

# Show that complex units are equivalent to Pa
print('P in Pa is', uc.get_in_units(p, 'Pa'))

P in working units is 1.2795093602644564e-09
P in Pa is 205.0


## 3. Manual creation of an atomic System

The atomic configurations represented in atomman consist of three primary classes:

- Box defines a system/cell box,
- Atoms manages per-atom properties, and 
- System combines the two to provide a representation of the full atomic system.

### 3.1. Creating a Box

A Box consists of four vectors
- avect is the Cartesian vector associated with the Box’s a lattice vector.
- bvect is the Cartesian vector associated with the Box’s b lattice vector.
- cvect is the Cartesian vector associated with the Box’s c lattice vector.
- origin is the Cartesian position vector for the Box’s origin. The positions of the Box’s eight corners are given by adding combinations of avect, bvect and cvect to the origin.

A new Box can be created by specifying the box dimensions in a number of formats
- vects (and origin): a 3x3 array of the three box vectors plus the optional origin
- avect, bvect, cvect (and origin): the three separate box vectors plus the optional origin.
- a, b, c, (alpha, beta, gamma and origin): the lattice constants plus the optional lattice angles and origin.  Any angle not given are assumed to be 90.
- xlo, xhi, ylo, yhi, zlo, zhi, (xy, xz and yz): The LAMMPS box bounds and optional tilts.
- lx, ly, lz, (xy, xz, yz, and origin): The LAMMPS box lengths and optional tilts and origin.

Cubic box for an FCC unit cell system for Al

In [2]:
a = uc.set_in_units(4.05, 'angstrom')

box = am.Box(a=a, b=a, c=a)
print(box)

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]


Alternatively, methods named for crystal families (cubic, hexagonal, etc) allow for boxes to be specified with only the unique lattice constants and angles.

In [3]:
box = am.Box.cubic(a=a)
print(box)

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]


### 3.2. Creating Atoms

Atoms objects manage per-atom properties for all of the atoms in a system.  The basic design of atomman.Atoms is that

- The number of atoms is constant.  Note that new Atoms objects can be constructed by copying/deleting atoms from other Atoms objects.
- The only required per-atom properties are a positive integer atype, and a 3D vector pos.
- Other per-atom properties can be freely assigned, as long as the property is shared by all atoms and is of the same data type and shape.

In [7]:
# Construct atoms with reduced coordinates for a face-centered cell
atoms = am.Atoms(atype=1, pos=[[0.0, 0.0, 0.0],
                               [0.5, 0.5, 0.0],
                               [0.5, 0.0, 0.5], 
                               [0.0, 0.5, 0.5]])
print(atoms)

per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   0.500 |   0.500 |   0.000
      2 |       1 |   0.500 |   0.000 |   0.500
      3 |       1 |   0.000 |   0.500 |   0.500


Per-atom properties can be set and retrieved as object attributes if the names are legal attribute names.

In [14]:
atoms.charge = [0.0, 0.0, 0.0, 0.0]
print(atoms.charge)

[0. 0. 0. 0.]


In [15]:
print(atoms.pos[2])

[0.5 0.  0.5]


Using indexing on the Atoms object itself will return a new Atoms object of the specified slice.

In [16]:
print(atoms[[0,2]])

per-atom properties = ['atype', 'pos', 'charge']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   0.500 |   0.000 |   0.500


The atomic properties can also be copied to a pandas.DataFrame representation.

In [18]:
atoms.df()

,atype,pos[0],pos[1],pos[2],charge
0,1,0.0,0.0,0.0,0.0
1,1,0.5,0.5,0.0,0.0
2,1,0.5,0.0,0.5,0.0
3,1,0.0,0.5,0.5,0.0


### 3.3. Creating Systems

A System object can then be created by combining Box and Atoms objects.

- setting scale=True will scale atomic pos by the box dimensions.
- symbols assigns atomic model symbols to the unique integer atype values.
- pbc allows for the three periodic boundary conditions to be set/changed, with True indicating that the associated box dimension is periodic

In [19]:
ucell = am.System(atoms=atoms, box=box, scale=True, symbols=['Al'])
print(ucell)

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
symbols = ('Al',)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos', 'charge']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.025 |   2.025 |   0.000
      2 |       1 |   2.025 |   0.000 |   2.025
      3 |       1 |   0.000 |   2.025 |   2.025


The Atoms and Box used to create the system can still be directly accessed for exploration and modifications.

In [33]:
print(ucell.box)

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]


In [34]:
print(ucell.atoms)

per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.500 |   0.500 |   0.500
      1 |       1 |   0.500 |   2.525 |   2.525
      2 |       1 |   2.525 |   0.500 |   2.525
      3 |       1 |   2.525 |   2.525 |   0.500


## 4. Loading an atomic configuration

An atomman.System can also be constructed by loading configuration information from other sources.

- From files or strings: POSCAR, CIF, LAMMPS data, LAMMPS dump, atomman JSON/XML format 
- From databases: crystal prototypes, NIST relaxed crystals, DFT relaxed crystals from Materials Project and/or OQMD
- From alternate Python representations: ase.Atoms, phonopy.Atoms, pymatgen.Structure, spglib.cell

### 4.1. Load a unit cell system from a POSCAR str

In [20]:
poscar="""POSCAR demo for fcc Al
4.0500000
1.000000 0.000000 0.000000
0.000000 1.000000 0.000000
0.000000 0.000000 1.000000
Al
4
Cartesian
0.000000 0.000000 0.000000 1 0 0
2.025000 2.025000 0.000000 1 0 0
2.025000 0.000000 2.025000 1 0 0
0.000000 2.025000 2.025000 1 0 0
"""
ucell = am.load('poscar', poscar)
print(ucell)

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
symbols = ('Al',)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   2.025 |   2.025 |   0.000
      2 |       1 |   2.025 |   0.000 |   2.025
      3 |       1 |   0.000 |   2.025 |   2.025


### 4.2. Load a LAMMPS potential and an associated relaxed crystal from the NIST database

In [21]:
potential = am.load_lammps_potential(id='2003--Zope-R-R--Al--LAMMPS--ipr1')

In [22]:
ucell = am.load('crystal', potential=potential, family='A1--Cu--fcc')
print(ucell)

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
symbols = ('Al',)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   0.000 |   2.025 |   2.025
      2 |       1 |   2.025 |   0.000 |   2.025
      3 |       1 |   2.025 |   2.025 |   0.000


## 5. Basic System modifications

### 5.1. Rigid shift

A rigid shift can simply be applied by adding/subtracting a constant from all atomic positions.


In [24]:
ucell.atoms.pos += [0.5, 0.5, 0.5]
print(ucell)

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
symbols = ('Al',)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.500 |   0.500 |   0.500
      1 |       1 |   0.500 |   2.525 |   2.525
      2 |       1 |   2.525 |   0.500 |   2.525
      3 |       1 |   2.525 |   2.525 |   0.500


### 5.2. Wrap atoms/bounds

The wrap() method will ensure all atom positions are inside the box by
- wrapping atoms past periodic boundaries back into the box
- shifting non-periodic boundaries to encompass all atoms 

In [25]:
ucell.wrap()
print(ucell)

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
symbols = ('Al',)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.500 |   0.500 |   0.500
      1 |       1 |   0.500 |   2.525 |   2.525
      2 |       1 |   2.525 |   0.500 |   2.525
      3 |       1 |   2.525 |   2.525 |   0.500


### 5.3. Change periodic boundary conditions

The periodic boundary conditions can be changed by setting new values to pbc.

In [27]:
ucell.pbc[2] = False
print(ucell.pbc)
ucell.pbc = [True, True, True]
print(ucell.pbc)

[ True  True False]
[ True  True  True]


### 5.4. Rotate

Systems can be rotated by specifying three Miller vectors relative to the current system's box vectors.  This ensures that the system remains coherent across any periodic boundary conditions.

In [29]:
rcell = ucell.rotate([[ 1, 1,-2],
                      [ 1, 1, 1],
                      [-1, 1, 0]])
print(rcell)

avect =  [ 9.920,  0.000,  0.000]
bvect =  [ 0.000,  7.015,  0.000]
cvect =  [ 0.000,  0.000,  5.728]
origin = [ 0.000,  0.000,  0.000]
natoms = 24
natypes = 1
symbols = ('Al',)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   7.440 |   0.866 |   1.432
      1 |       1 |   7.440 |   0.866 |   4.296
      2 |       1 |   9.094 |   3.204 |   4.296
      3 |       1 |   9.094 |   3.204 |   1.432
      4 |       1 |   2.480 |   0.866 |   4.296
      5 |       1 |   4.960 |   0.866 |   5.728
      6 |       1 |   2.480 |   0.866 |   1.432
      7 |       1 |   4.960 |   0.866 |   2.864
      8 |       1 |   4.134 |   3.204 |   1.432
      9 |       1 |   4.134 |   3.204 |   4.296
     10 |       1 |   6.614 |   3.204 |   2.864
     11 |       1 |   6.614 |   3.204 |   0.000
     12 |       1 |   5.787 |   5.543 |   4.296
     13 |       1 |   8.267 |   5.543 |   0.000
     14 |       1 |   5.787 |   5.543

### 5.5. Replicate

The supersize method replicates the current system along each of the box vectors a specified number of times. Replications can be positive, negative or a combination of the two.  For negative replications, the system is shifted/copied in the negative Cartesian direction.

In [30]:
# Create 2x2x2 supercell, where a is replicated in the positive direction, b in the negative direction, and c equally in both directions 
bigsystem = ucell.supersize(2, -2, (-1, 1))
print(bigsystem)

avect =  [ 8.100,  0.000,  0.000]
bvect =  [ 0.000,  8.100,  0.000]
cvect =  [ 0.000,  0.000,  8.100]
origin = [ 0.000, -8.100, -4.050]
natoms = 32
natypes = 1
symbols = ('Al',)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.500 |  -7.600 |  -3.550
      1 |       1 |   0.500 |  -5.575 |  -1.525
      2 |       1 |   2.525 |  -7.600 |  -1.525
      3 |       1 |   2.525 |  -5.575 |  -3.550
      4 |       1 |   4.550 |  -7.600 |  -3.550
      5 |       1 |   4.550 |  -5.575 |  -1.525
      6 |       1 |   6.575 |  -7.600 |  -1.525
      7 |       1 |   6.575 |  -5.575 |  -3.550
      8 |       1 |   0.500 |  -3.550 |  -3.550
      9 |       1 |   0.500 |  -1.525 |  -1.525
     10 |       1 |   2.525 |  -3.550 |  -1.525
     11 |       1 |   2.525 |  -1.525 |  -3.550
     12 |       1 |   4.550 |  -3.550 |  -3.550
     13 |       1 |   4.550 |  -1.525 |  -1.525
     14 |       1 |   6.575 |  -3.550

### 5.6. Slice and extend

- System.atoms_ix[] creates a new system with same box, pbc info with a slice of atoms.  
- System.atoms_extend() creates a new system with the same box, pbc info but with extra atoms added in. Can either specify an int number of new atoms to add, or an Atoms object. 

In [31]:
print(bigsystem.atoms_ix[:4])

avect =  [ 8.100,  0.000,  0.000]
bvect =  [ 0.000,  8.100,  0.000]
cvect =  [ 0.000,  0.000,  8.100]
origin = [ 0.000, -8.100, -4.050]
natoms = 4
natypes = 1
symbols = ('Al',)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.500 |  -7.600 |  -3.550
      1 |       1 |   0.500 |  -5.575 |  -1.525
      2 |       1 |   2.525 |  -7.600 |  -1.525
      3 |       1 |   2.525 |  -5.575 |  -3.550


In [32]:
print(ucell.atoms_extend(4))

avect =  [ 4.050,  0.000,  0.000]
bvect =  [ 0.000,  4.050,  0.000]
cvect =  [ 0.000,  0.000,  4.050]
origin = [ 0.000,  0.000,  0.000]
natoms = 8
natypes = 1
symbols = ('Al',)
pbc = [ True  True  True]
per-atom properties = ['atype', 'pos']
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.500 |   0.500 |   0.500
      1 |       1 |   0.500 |   2.525 |   2.525
      2 |       1 |   2.525 |   0.500 |   2.525
      3 |       1 |   2.525 |   2.525 |   0.500
      4 |       1 |   0.000 |   0.000 |   0.000
      5 |       1 |   0.000 |   0.000 |   0.000
      6 |       1 |   0.000 |   0.000 |   0.000
      7 |       1 |   0.000 |   0.000 |   0.000


## 6. System analysis tools

### 6.1. Shortest interatomic spacing

The System.r0() method will compute the shortest atomic spacing.  Note that this checks *all* atoms, so perform it on systems before supersizing if possible)

In [35]:
ucell.r0()

2.863782440859508

### 6.2. Relative distance vectors and magnitudes

The System.dvect() and System.dmag() methods compute the shortest relative distance vectors and magnitudes between atoms or arbitrary points by considering periodic boundary conditions.

In [37]:
# Compute dvects between all atoms in ucell and atom with index 2 (the third atom)
ucell.dvect(2, range(4))

array([[-2.02499998,  0.        , -2.02499998],
       [-2.02499998,  2.02499998,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  2.02499998, -2.02499998]])

In [40]:
# Compute dmags between all atoms in ucell and position [1.0, 1.0, 1.0]
ucell.dmag([1.0, 1.0, 1.0], range(4))

array([0.8660254 , 2.21387667, 2.21387667, 2.21387667])

### 6.3. Neighbor lists

The System.neighborlist() method identifies all nearest neighbor atoms for all atoms within a specified cutoff.  

__NOTE__: The underlying algorithm uses dmag, and therefore when assigning neighbors *does not* differentiate between multiple replicas of the same atom.  As a consequence, the number of neighbors may be undercounted if any atoms neighbor multiple periodic replicas of the same atom.

In [42]:
testsystem = ucell.supersize(3,3,3)
nlist = testsystem.neighborlist(cutoff = ucell.r0()*1.2)

coord returns the coordination count for each atom.

In [43]:
nlist.coord.mean()

12.0

The indices of neighbors for a given atom can be easily retrieved.

In [46]:
nlist[0]

array([ 1,  2,  3, 10, 11, 25, 27, 35, 73, 74, 82, 97], dtype=int64)